In [ ]:
import json

import torch as t

from challenge_gorfoustral.gorfougym import load_model_vague_2

In [2]:
model = load_model_vague_2(device="cuda", dtype=t.float16)

Loaded pretrained model tiny-stories-instruct-33M into HookedTransformer


In [3]:
with open("challenge_tatouage/journal35.json", "r") as f:
    data = json.load(f)

len(data)

180

## 1ère étape : Trouver les phrases tatouées

Pour ça on peut utiliser la perplexité. C'est une mesure qui donne la probabilité que la phrase aie été générée par le modèle. Si la phrase est tatouée, la perplexité augmente car les jetons choisis ne sont plus forcément dans les premiers choix. 

Pour GPT2 et la méthode de tatouage proposée, la perplexité permet de distinguer la majorité des cas : 

![ppls](../assets/ppls.png)

Pour faciliter le challenge, j'ai selectionné les phrases de telle sorte que toutes les perplexitées des phrases tatouées soient plus élevées que toutes les perplexitées des phrases non tatouées. 

In [4]:
ppls = []

for x in data:
    tokens = model.to_tokens(x)
    loss = model.forward(tokens, return_type="loss")
    ppls.append((loss.item(), x))


ppls_ranked = sorted(ppls, key=lambda x: x[0], reverse=True)
watermarked_sentences = [x[1] for x in ppls_ranked[: len(ppls_ranked) // 2]]

## 2ème étape : Trouver des suites de jetons récurrentes

L'idée est très simple, si l'on a plusieurs fois la suite a-b-x avec plusieurs x différents, alors on peut essayer de reconstruire le masque utilisé. 

In [8]:
d = {}
start_idx = 20
mask_len = 10

for prompt in watermarked_sentences:
    tokens = model.to_tokens(prompt)[0]
    for i in range(start_idx, len(tokens) - 2):
        d[tokens[i].item(), tokens[i + 1].item()] = d.get(
            (tokens[i].item(), tokens[i + 1].item()), set()
        ).union(set([tokens[i + 2].item()]))

# Filtre sur les paires qui apparaissent au moins mask_len fois, sinon aucune chance
# d'avoir le masque en entier
dd = {}
for k, v in d.items():
    if len(v) > mask_len:
        dd[k] = v

jetons_récurrents = [(k[0], k[1], list(v), len(v)) for k, v in dd.items()]
jetons_récurrents = sorted(jetons_récurrents, key=lambda x: x[-1], reverse=True)

## 3ème étape : Reconstruction du masque

Une fois les jetons récurrents trouvés, il faut extraire le début des phrases, tout ce qui est entre 20 (début du tatouage) et la séquence a-b-x, pour pouvoir reformer le masque.

In [ ]:
potentiels_masques = []

for r in jetons_récurrents:
    tok1, tok2, _, _ = r
    masque = []
    generations = []

    for x in watermarked_sentences:
        tokens = model.to_tokens(x)[0]
        parts = []
        for i in range(start_idx, len(tokens) - 2):
            if tokens[i] == tok1 and tokens[i + 1] == tok2:
                parts.append(tokens[: i + 3])

        if len(parts) > 0:
            generations.append(parts)

    for x in generations:
        for y in x:
            tokens, next_tok = y[:-1], y[-1]
            logits = model(tokens)
            _, indices = logits[0, -1, :].topk(k=2 * mask_len)

            try:
                finder = (indices == next_tok).nonzero()
                masque.append(finder[0][0].item())
            except:
                pass

    if len(set(masque)) == mask_len:
        potentiels_masques.append((masque, *r))

## 4ème étape : Récupération de la seed

Une fois le masque récupéré, il suffit d'énumérer le "cryptage très puissant" pour récupérer la seed. 

In [ ]:
from challenge_tatouage.tatouage import get_mask


def get_seed_from_mask(mask_orig, mask_len: int):
    results = []
    for i in range(99999):
        if t.equal(mask_orig, get_mask(i, mask_len=mask_len)):
            results.append(i)
    return results


results = []
for potentiel_masque in potentiels_masques:
    generation, tok1, tok2, _, _ = potentiel_masque

    mask = t.zeros(mask_len * 2, dtype=t.bool)
    mask[list(set(generation))] = True
    seeds = get_seed_from_mask(mask, mask_len=mask_len)

    for seed in seeds:
        for i in range(50033):
            if (tok2 * i + tok1) % 50033 == seed:
                results.append(i)

results

[43633]